length of enc will be 1 shorter if last 3 of same chars matches 4nd last char OR last 3 chars are null

In [2]:
from string import ascii_lowercase, ascii_uppercase
from pwnlib.tubes.remote import remote
from tqdm import tqdm
from ast import literal_eval
import zlib
import re
from random import randint
import os
from Crypto.Cipher import Salsa20
secret = os.urandom(32)
cipher = Salsa20.new(key=secret)
flag = os.urandom(45)

In [99]:
r = remote('mercury.picoctf.net', 50899)
def encrypt(p: bytes):
    r.recvuntil(b"Enter your text to be encrypted: ").decode()
    r.send(p+b"\n")
    nonce = literal_eval(r.recvline().decode())
    c = literal_eval(r.recvline().decode())
    len_c = int(r.recvline().decode())
    return nonce, c, len_c

In [ ]:
alph = "".join([chr(c) for c in range(33,128)])#ascii_lowercase+"0123456789_{}"
flag = b""
leak_starter = []
for c in tqdm(alph):
    leak_starter.append(encrypt(c.encode()*3)[2])
flag = bytes([33+leak_starter.index(46)])+flag

In [100]:
flag = b"riff_you_solved_the_crime}"
alph = "".join([chr(c) for c in range(33,128)])#ascii_lowercase+"0123456789_{}"
# bounds: start a padding ensures it works for all (parity check fails for bigger length)
# flag is also cropped (to 8 max) because the parity check fails otherwise
# don't need to do the first thing if we do the 2nd
# scratch that, do any of the above 2 methods until the program doesn't crash
leak = []
for i in range(1):
    leak_char = []  
    for c in tqdm(alph):
        leak_char.append(encrypt((c.encode()+flag[:8]))[2])
    # for c in flag:
    #     leak_char[c-33] = 50
    leak.append(leak_char)
    flag = bytes([33+leak_char.index(47)])+flag
# [len(zlib.compress(flag+bytes([c, 84]*2)))==57 for c in range(0,256)].index(False)
# [len(zlib.compress(flag+bytes([c]*3)))==59 for c in range(0,256)].index(False)
# encrypt("a")

100%|██████████| 95/95 [01:03<00:00,  1.50it/s]


In [71]:
chr(leak_char.index(48))

'\x00'

In [101]:
flag

b'eriff_you_solved_the_crime}'

In [41]:
chr(33+leak_char.index(46))

'o'

In [241]:
import pandas as pd
pd.Series(leak_char).sort_values(ascending=True)[:10]

83    47
0     48
2     48
1     48
4     48
5     48
6     48
3     48
8     48
9     48
dtype: int64

In [20]:
[len(zlib.compress(b"cblved_the"+b"c"+bytes([c])+b"lved_the")) for c in range(33, 127)]

[21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21]

In [30]:
print(len(zlib.compress(b"cblved_the_crime}"+b"cblved_the")))
print(len(zlib.compress(b"cblved_the_crime}"+b"calved_the")))
print(len(zlib.compress(b"cblved_the_crime}"+b"dblved_the")))
print(len(zlib.compress(b"cblved_the_crime}"+b"dalved_the")))

28
29
28
29


In [155]:
[len(zlib.compress(flag+bytes([c, 99, 114, 105, 109, 101, 125]*2))) for c in range(0,128)]

[17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17]

In [154]:
list(flag)

[95, 99, 114, 105, 109, 101, 125]

In [121]:
len(zlib.compress(flag[6:]+(bytes([ord('_')])+flag[6:])*2))

13

In [122]:
len(zlib.compress(flag[6:]+(bytes([ord('k')])+flag[6:])*2))

13

In [110]:
[len(zlib.compress(flag[1:]+(bytes([c])+flag[1:])*2)) for c in range(0,128)]

ValueError: 15 is not in list

In [97]:
[len(zlib.compress(b"a"+flag+(bytes([c])+flag)*3)) for c in range(0,256)]

[19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,


In [98]:
len(zlib.compress(b"a"+flag+(bytes([0])+flag)*2))

18

In [92]:
chr(144)

'\x90'

In [69]:
[len(zlib.compress(flag+bytes([c, 84]*2)))==57 for c in range(0,256)].index(False)

0

In [64]:
len(zlib.compress(bytes(("ime}"+("ime}")*2).encode("utf-8"))))

14

In [48]:
[len(zlib.compress(bytes(("_crime}"+(chr(i)+"crime}")*2).encode("utf-8")))) for i in range(0,128)]

[17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17]

In [34]:
len(zlib.compress(bytes("pico}}}Cgewg}}}wgwgwh}whwTF{}}}}}}}}}}}}}}}}}}}}}}}}}".encode("utf-8"))))

36

In [ ]:
encrypt((c.encode()+b"\x80")*2)

In [ ]:
temp2 = []
for c in tqdm([chr(i) for i in range(0,256)]):
    temp2.append(len(encrypt((c.encode()+b"\x80")*2)[1]))

In [17]:
[len(zlib.compress(flag+bytes([16]*i))) for i in range(1,10)]

[55, 56, 56, 56, 56, 56, 56, 56, 56]

In [24]:
len(zlib.compress(flag+bytes([0]*3)))

57

In [22]:
[len(zlib.compress(flag+bytes([c]*3)))==56 for c in range(0,256)].index(True)

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 

In [12]:
[len(zlib.compress(flag+bytes([c]*3)))==57 for c in range(0,256)].index(False)

16

In [3]:
temp.index(46)

128

In [ ]:
flag = "???T"

In [93]:
zip_len = len(zlib.compress(flag+bytes([0]*3)))

59

In [94]:
[len(zlib.compress(flag+bytes([c, 60, 82, 84]*2)))==57 for c in range(0,256)].index(False)

27

In [88]:
[len(zlib.compress(flag+bytes([c, 82, 84]*2)))==57 for c in range(0,256)].index(False)

60

In [91]:
[len(zlib.compress(flag+bytes([c, 84]*2)))==57 for c in range(0,256)].index(False)

82

In [90]:
[len(zlib.compress(flag+bytes([c]*3)))==59 for c in range(0,256)].index(False)

84

In [11]:
cipher.encrypt(b"\x00"*45)

45